In [23]:
import numpy as np
import pandas as pd
df = pd.read_csv("../data/单价按日期单品合并_剔除后.csv",encoding='utf-8')
print(df)

           销售日期  102900005115823   102900005115908   102900005115779   \
0      2020/7/1         10.000000               8.0          8.000000   
1      2020/7/2         10.000000               8.0          7.958559   
2      2020/7/3          9.858621               8.0          8.000000   
3      2020/7/4         10.000000               8.0          8.000000   
4      2020/7/5          9.989286               8.0          7.997826   
...         ...               ...               ...               ...   
1090  2023/6/26          8.000000               0.0          9.200000   
1091  2023/6/27          8.000000               6.0          9.200000   
1092  2023/6/28          8.000000               6.0          0.000000   
1093  2023/6/29          8.000000               0.0          9.200000   
1094  2023/6/30          8.000000               0.0          0.000000   

      102900005118824   102900005115984   102900005115762   102900005116714   \
0            10.000000           6.00000   

In [24]:
pl = pd.read_excel("../data/附件1.xlsx")
pl['单品编码'] = pl['单品编码'].astype(str)
pl = pl.set_index('单品编码')

In [25]:
# LSTM for international airline passengers problem with regression framing
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
tf.random.set_seed(7)

In [ ]:
predict = pd.DataFrame([])
RMSE = pd.DataFrame([])
for index in df:
	if index == '销售日期':
		continue
	# load the dataset
	name_CN = pl.loc[index[:-1]]['单品名称']
	print(name_CN)
	dataframe = df[['销售日期',index]]
	dataframe.set_index(["销售日期"], inplace=True)
	dataset = dataframe.values
	dataset = dataset.astype('float32')
	# normalize the dataset
	scaler = MinMaxScaler(feature_range=(0, 1))
	dataset = scaler.fit_transform(dataset)
	# split into train and test sets
	train_size = int(len(dataset) * 0.99)
	test_size = len(dataset) - train_size
	train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
	# reshape into X=t and Y=t+1
	look_back = 7
	trainX, trainY = create_dataset(train, look_back)
	testX, testY = create_dataset(test, look_back)
	# reshape input to be [samples, time steps, features]
	trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
	testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
	# create and fit the LSTM network
	model = Sequential()
	model.add(LSTM(30, input_shape=(1, look_back)))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	model.fit(trainX, trainY, epochs=30, batch_size=1, verbose=2)
	# make predictions
	trainPredict = model.predict(trainX)
	testPredict = model.predict(testX)
	# invert predictions
	trainPredict = scaler.inverse_transform(trainPredict)
	trainY = scaler.inverse_transform([trainY])
	testPredict = scaler.inverse_transform(testPredict)
	testY = scaler.inverse_transform([testY])
	# calculate root mean squared error
	trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
	print('Train Score: %.2f RMSE' % (trainScore))
	testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
	print('Test Score: %.2f RMSE' % (testScore))
	RMSE[name_CN] = [trainScore]
	print(trainScore)
	# shift train predictions for plotting
	trainPredictPlot = np.empty_like(dataset)
	trainPredictPlot[:, :] = np.nan
	trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
	# shift test predictions for plotting
	testPredictPlot = np.empty_like(dataset)
	testPredictPlot[:, :] = np.nan
	testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
	# plot baseline and predictions
	plt.plot(scaler.inverse_transform(dataset))
	plt.plot(trainPredictPlot)
	plt.plot(testPredictPlot)
	plt.show()

	testFinal = dataset[-look_back:]
	testFinal = [pd.DataFrame(testFinal).T.values.tolist()]
	testFinalPredict = model.predict(testFinal)
	testFinalPredict = scaler.inverse_transform(testFinalPredict)
	print(testFinalPredict)
	predict[name_CN] = testFinalPredict[0]
	print("--------------------------")

In [27]:
print(RMSE)
print(predict)

        上海青       菜心      云南生菜     高瓜(1)     云南油麦菜        苋菜       西兰花  \
0  1.544887  2.40101  1.951617  3.870446  1.476913  2.414915  1.423895   

        菠菜      洪湖藕带       红薯尖  ...     高瓜(2)  姜蒜小米椒组合装(小份)    双孢菇(盒)  \
0  3.04003  5.927016  2.353551  ...  4.211677      0.709058  0.801868   

     枝江青梗散花    鲜木耳(份)  青红杭椒组合装(份)    紫茄子(1)     红椒(2)  蟹味菇与白玉菇双拼(盒)    木耳菜(份)  
0  1.177407  0.427737    0.859117  0.908299  2.555565      1.205094  0.226906  

[1 rows x 49 columns]
        上海青       菜心      云南生菜      高瓜(1)    云南油麦菜        苋菜        西兰花  \
0  7.807025  2.40725  2.765698  13.566231  1.24095  3.005982  10.383276   

         菠菜       洪湖藕带       红薯尖  ...      高瓜(2)  姜蒜小米椒组合装(小份)    双孢菇(盒)  \
0  2.986271  18.125647  4.603646  ...  12.549412      3.734741  4.671286   

     枝江青梗散花    鲜木耳(份)  青红杭椒组合装(份)    紫茄子(1)      红椒(2)  蟹味菇与白玉菇双拼(盒)    木耳菜(份)  
0  8.854301  0.210509    4.798176  0.732851  17.867697      3.519409  4.296689  

[1 rows x 49 columns]


In [28]:
RMSE.to_csv('../data/单价RMSE.csv',sep=',',encoding='utf_8_sig')
predict.to_csv('../data/单价7月1日预估.csv',sep=',',encoding='utf_8_sig')